First we define a couple of functions that we will need later, for formating and printing alignments

In [10]:
import argparse
import numpy as np

def print_alignment(seqA,seqB):
  print(seqA)
  print(seqB)

def print_dynamic(seqA,seqB,dpm):
  seqA,seqB = "-" + seqA, "-" + seqB
  m,n = len(seqA),len(seqB)
  print '{:^5}'.format(" "),
  for j in range(n):
    print '{:^5}'.format(seqB[j]),
  print
  for i in range(m):
    print '{:^5}'.format(seqA[i]),
    for j in range(n):
        print '{:5.1f}'.format(dpm[i,j]),
    print
  print

def format_alignment(seqA,seqB,trace):
  outA,outB = "",""
  i,j = len(seqA),len(seqB)
  while i>0 or j>0:
    di,dj = trace[i,j]
    i += int(di)
    j += int(dj)
    if di == 0:
      outA = "-" + outA
    else:
      outA = seqA[i] + outA
    if dj == 0:
      outB = "-" + outB
    else:
      outB = seqB[j] + outB
  return outA,outB

Then we setup the scoring system we need for the alignment

In [11]:
gap_penalty = -1.0

def match_score(letterA,letterB):
  if letterA == letterB:
    return 1.0
  else:
    return -1.0


Now we turn our attention to the core of the Needleman-Wunsch, the dynamic programming

In [15]:
def align(seqA,seqB,print_dynamic_matrix = False):
    # Initiating variables
    m, n = len(seqA)+1, len(seqB)+1
    S = np.zeros((m,n))
    trace = np.zeros((m,n,2))
    # Set up dynamic programming matrices
    S[0,0] = 0.
    trace[0,0,:] = (0.,0.)
    for i in range(1,m):
        S[i,0] = i * gap_penalty
        trace[i,0,:] =(-1,0)
    for j in range(1,n):
        S[0,j] = j * gap_penalty
        trace[0,j,:] =(0,-1)
    for i in range(1,m):
        for j in range(1,n):
            # Note the subtraction of 1 for the sequence position
            #
            match = S[i-1,j-1] + match_score(seqA[i-1],seqB[j-1]) 
            delete = S[i-1,j] + gap_penalty
            insert = S[i,j-1] + gap_penalty
            S[i,j] = max(match,delete,insert)
            if match >= max(insert,delete):
                trace[i,j,:] = (-1,-1)
            elif delete >= insert:
                trace[i,j,:] = (-1,0)
            else:
                trace[i,j,:] = (0,-1)
    if print_dynamic_matrix:
        print_dynamic(seqA,seqB,S)
    print("Best score: " + str(S[m-1,n-1]))
    return format_alignment(seqA,seqB,trace)

Now everything is set. We can try the code for any of our favorite sequences. One can toggle the printout of the dynamic programming matrix by a boolean flag as a third argument.

In [16]:
seqA,seqB = align("GATTA","GCTAC",True)
print_alignment(seqA,seqB)

        -     G     C     T     A     C  
  -     0.0  -1.0  -2.0  -3.0  -4.0  -5.0
  G    -1.0   1.0   0.0  -1.0  -2.0  -3.0
  A    -2.0   0.0   0.0  -1.0   0.0  -1.0
  T    -3.0  -1.0  -1.0   1.0   0.0  -1.0
  T    -4.0  -2.0  -2.0   0.0   0.0  -1.0
  A    -5.0  -3.0  -3.0  -1.0   1.0   0.0

Best score: 0.0
GATTA-
G-CTAC


In [17]:
seqA,seqB = align("GCGATTA","GCTTAC",True)
print_alignment(seqA,seqB)

        -     G     C     T     T     A     C  
  -     0.0  -1.0  -2.0  -3.0  -4.0  -5.0  -6.0
  G    -1.0   1.0   0.0  -1.0  -2.0  -3.0  -4.0
  C    -2.0   0.0   2.0   1.0   0.0  -1.0  -2.0
  G    -3.0  -1.0   1.0   1.0   0.0  -1.0  -2.0
  A    -4.0  -2.0   0.0   0.0   0.0   1.0   0.0
  T    -5.0  -3.0  -1.0   1.0   1.0   0.0   0.0
  T    -6.0  -4.0  -2.0   0.0   2.0   1.0   0.0
  A    -7.0  -5.0  -3.0  -1.0   1.0   3.0   2.0

Best score: 2.0
GCGATTA-
GC--TTAC


In [18]:
seqA,seqB = align("GCTATCTCGCTA","GCTAGCTA",True)
print_alignment(seqA,seqB)

        -     G     C     T     A     G     C     T     A  
  -     0.0  -1.0  -2.0  -3.0  -4.0  -5.0  -6.0  -7.0  -8.0
  G    -1.0   1.0   0.0  -1.0  -2.0  -3.0  -4.0  -5.0  -6.0
  C    -2.0   0.0   2.0   1.0   0.0  -1.0  -2.0  -3.0  -4.0
  T    -3.0  -1.0   1.0   3.0   2.0   1.0   0.0  -1.0  -2.0
  A    -4.0  -2.0   0.0   2.0   4.0   3.0   2.0   1.0   0.0
  T    -5.0  -3.0  -1.0   1.0   3.0   3.0   2.0   3.0   2.0
  C    -6.0  -4.0  -2.0   0.0   2.0   2.0   4.0   3.0   2.0
  T    -7.0  -5.0  -3.0  -1.0   1.0   1.0   3.0   5.0   4.0
  C    -8.0  -6.0  -4.0  -2.0   0.0   0.0   2.0   4.0   4.0
  G    -9.0  -7.0  -5.0  -3.0  -1.0   1.0   1.0   3.0   3.0
  C   -10.0  -8.0  -6.0  -4.0  -2.0   0.0   2.0   2.0   2.0
  T   -11.0  -9.0  -7.0  -5.0  -3.0  -1.0   1.0   3.0   2.0
  A   -12.0 -10.0  -8.0  -6.0  -4.0  -2.0   0.0   2.0   4.0

Best score: 4.0
GCTATCTCGCTA
GCTA----GCTA
